<a href="https://colab.research.google.com/github/hirusha-234/HLS-Convert-/blob/main/HLS_Conver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===========================================
# 🔧 Step 1: Setup (with GPU Detection)
# ===========================================
from google.colab import drive
import os, subprocess

# Mount Google Drive
drive.mount('/content/drive')

# Base paths in your Google Drive
drive_root = "/content/drive/MyDrive"
input_path = os.path.join(drive_root, "input_videos")
output_path = os.path.join(drive_root, "output_hls")

# Create input/output folders if they don't exist
os.makedirs(input_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

print(f"✅ Input folder created at: {input_path}")
print(f"✅ Output folder created at: {output_path}")

# ===========================================
# ⚙️ GPU Check
# ===========================================
gpu_info = subprocess.run("nvidia-smi --query-gpu=name --format=csv,noheader", shell=True, capture_output=True, text=True)

if gpu_info.returncode == 0 and gpu_info.stdout.strip():
    gpu_name = gpu_info.stdout.strip()
    print(f"\n💪 GPU Detected: {gpu_name}")
    print("Installing FFmpeg with NVIDIA GPU (NVENC) support...")
    !apt-get update -qq
    !apt-get install -y ffmpeg
    USE_GPU = True
else:
    print("\n⚠️ No GPU detected or incompatible runtime. Using CPU encoding (slower).")
    !apt-get update -qq
    !apt-get install -y ffmpeg
    USE_GPU = False

print("\n👉 Place your .mp4 file(s) in the 'input_videos' folder in your Drive, then run the next cell.")


Mounted at /content/drive
✅ Input folder created at: /content/drive/MyDrive/input_videos
✅ Output folder created at: /content/drive/MyDrive/output_hls

💪 GPU Detected: Tesla T4
Installing FFmpeg with NVIDIA GPU (NVENC) support...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.

👉 Place your .mp4 file(s) in the 'input_videos' folder in your Drive, then run the next cell.


In [ ]:
import subprocess, time, os, sys

# Auto-select encoder based on GPU availability
encoder = "h264_nvenc" if "USE_GPU" in globals() and USE_GPU else "libx264"
print(f"⚙️ Using encoder: {encoder}")

# Get list of mp4 files in input folder
video_files = [f for f in os.listdir(input_path) if f.lower().endswith(".mp4")]

if not video_files:
    print("⚠️ No .mp4 files found in your Google Drive 'input_videos' folder.")
else:
    for file in video_files:
        file_path = os.path.join(input_path, file)
        basename = os.path.splitext(file)[0]
        output_dir = os.path.join(output_path, basename)
        os.makedirs(output_dir, exist_ok=True)

        print(f"\n🎞️==============================")
        print(f"🎬 Starting HLS Processing for: {file}")
        print(f"📁 Input: {file_path}")
        print(f"📂 Output folder: {output_dir}")
        print(f"🕒 Started at: {time.strftime('%H:%M:%S')}")
        print(f"==============================\n")

        # Build ffmpeg command with selected encoder
        ffmpeg_cmd = f"""
        ffmpeg -hide_banner -y -i "{file_path}" -filter_complex "
        [0:v]split=6[v144][v240][v360][v480][v720][v1080];
        [v144]scale=-2:144[v144out];
        [v240]scale=-2:240[v240out];
        [v360]scale=-2:360[v360out];
        [v480]scale=-2:480[v480out];
        [v720]scale=-2:720[v720out];
        [v1080]scale=-2:1080[v1080out]
        " \
        -map [v144out] -c:v:0 {encoder} -preset fast -b:v:0 150k -c:a aac -b:a 64k -f hls -hls_time 6 -hls_playlist_type vod "{output_dir}/144p.m3u8" \
        -map [v240out] -c:v:1 {encoder} -preset fast -b:v:1 400k -c:a aac -b:a 64k -f hls -hls_time 6 -hls_playlist_type vod "{output_dir}/240p.m3u8" \
        -map [v360out] -c:v:2 {encoder} -preset fast -b:v:2 800k -c:a aac -b:a 128k -f hls -hls_time 6 -hls_playlist_type vod "{output_dir}/360p.m3u8" \
        -map [v480out] -c:v:3 {encoder} -preset fast -b:v:3 1200k -c:a aac -b:a 128k -f hls -hls_time 6 -hls_playlist_type vod "{output_dir}/480p.m3u8" \
        -map [v720out] -c:v:4 {encoder} -preset fast -b:v:4 2500k -c:a aac -b:a 128k -f hls -hls_time 6 -hls_playlist_type vod "{output_dir}/720p.m3u8" \
        -map [v1080out] -c:v:5 {encoder} -preset fast -b:v:5 5000k -c:a aac -b:a 192k -f hls -hls_time 6 -hls_playlist_type vod "{output_dir}/1080p.m3u8"
        """

        print("🚀 Starting FFmpeg conversion...\n")

        # Run ffmpeg and show live progress
        process = subprocess.Popen(ffmpeg_cmd, shell=True, stderr=subprocess.PIPE, text=True)
        for line in iter(process.stderr.readline, ''):
            if "frame=" in line or "time=" in line or "bitrate=" in line:
                sys.stdout.write("\r" + line.strip())
                sys.stdout.flush()
        process.wait()

        print("\n✅ FFmpeg finished encoding all resolutions.")

        # Create master playlist
        master_playlist = """#EXTM3U
#EXT-X-VERSION:3

# 144p
#EXT-X-STREAM-INF:BANDWIDTH=150000,RESOLUTION=256x144
144p.m3u8

# 240p
#EXT-X-STREAM-INF:BANDWIDTH=400000,RESOLUTION=426x240
240p.m3u8

# 360p
#EXT-X-STREAM-INF:BANDWIDTH=800000,RESOLUTION=640x360
360p.m3u8

# 480p
#EXT-X-STREAM-INF:BANDWIDTH=1200000,RESOLUTION=854x480
480p.m3u8

# 720p
#EXT-X-STREAM-INF:BANDWIDTH=2500000,RESOLUTION=1280x720
720p.m3u8

# 1080p
#EXT-X-STREAM-INF:BANDWIDTH=5000000,RESOLUTION=1920x1080
1080p.m3u8
"""
        with open(os.path.join(output_dir, "master.m3u8"), "w") as f:
            f.write(master_playlist)

        print(f"\n📄 Master playlist created at: {output_dir}/master.m3u8")
        print(f"✅ Completed processing: {file}")
        print(f"📦 Output saved to Drive → output_hls/{basename}")
        print(f"⏱️ Finished at: {time.strftime('%H:%M:%S')}")
        print(f"==============================\n")

print("\n🎉 ALL DONE! Check your Google Drive → 'output_hls' folder.")


⚙️ Using encoder: h264_nvenc

🎞️==============================
🎬 Starting HLS Processing for: input_video.mp4
📁 Input: /content/drive/MyDrive/input_videos/input_video.mp4
📂 Output folder: /content/drive/MyDrive/output_hls/input_video
🕒 Started at: 18:34:29

🚀 Starting FFmpeg conversion...

frame= 3963 fps=5.2 q=22.0 Lq=-1.0 q=-1.0 q=-1.0 q=-1.0 q=-1.0 size=N/A time=00:01:06.09 bitrate=N/A speed=0.087x
✅ FFmpeg finished encoding all resolutions.

📄 Master playlist created at: /content/drive/MyDrive/output_hls/input_video/master.m3u8
✅ Completed processing: input_video.mp4
📦 Output saved to Drive → output_hls/input_video
⏱️ Finished at: 18:47:12


🎉 ALL DONE! Check your Google Drive → 'output_hls' folder.


In [ ]:
# ===========================================
# 📥 Download video from Google Drive into /input_videos
# ===========================================
import os, gdown

# Define target folder and file path
drive_root = "/content/drive/MyDrive"
input_path = os.path.join(drive_root, "input_videos")
os.makedirs(input_path, exist_ok=True)

# Google Drive share link
file_url = "https://drive.google.com/uc?id=1O0NmMhd4D9OhabU7hk-ZOs2tNCEYDZHO"
output_file = os.path.join(input_path, "input_video.mp4")

# Download using gdown
print("⬇️ Downloading video to your Drive input_videos folder...")
gdown.download(file_url, output_file, quiet=False)

print(f"✅ Saved to: {output_file}")


⬇️ Downloading video to your Drive input_videos folder...


Downloading...
From (original): https://drive.google.com/uc?id=1O0NmMhd4D9OhabU7hk-ZOs2tNCEYDZHO
From (redirected): https://drive.google.com/uc?id=1O0NmMhd4D9OhabU7hk-ZOs2tNCEYDZHO&confirm=t&uuid=019a172e-38ad-4d63-a86a-084f0075af01
To: /content/drive/MyDrive/input_videos/input_video.mp4
100%|██████████| 333M/333M [00:06<00:00, 48.6MB/s]

✅ Saved to: /content/drive/MyDrive/input_videos/input_video.mp4
